In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import MetagenomicData.filteringCuratedMetagenomicDataMetadata as fmd
import MetagenomicData.Plotting as pltmg
import MetagenomicData.Rankabundanceplot as ra


In [ ]:
metadata_file = "C:/Users/spigno/Documents/GitHub/PopulationData/MetagenomicData/MetadataMetgendata.csv"  # Percorso file metadati
metagenomic_data_file = "C:/Users/spigno/Documents/GitHub/PopulationData/MetagenomicData/RelabMetgendata.csv"  # Percorso file dati metagenomici
# Caricare i dataset
metadata_csv = pd.read_csv(metadata_file)
metagenomic_data = pd.read_csv(metagenomic_data_file)

In [ ]:
# Initialize the filter
metadata_filter = fmd.MetadataFilter(metadata_csv)

# Apply the filter criteria
filtered_metadata = metadata_filter.filter_data(
    diseases=["healthy"], 
    body_sites=["stool"],
    age_categories=["adult"],
    study_name=["DeFilippisF_2019"],
    diets=["omnivore", "vegetarian", "vegan"],
    )

# Exclude "healthy" manually
#filtered_metadata = filtered_metadata[filtered_metadata["disease"] != "healthy"]
filtered_metadata = filtered_metadata[filtered_metadata["antibiotics_current_use"] != "yes"]

In [23]:
# Obtaining the columns of the metagenomic dataset that correspond to the filtered metadata
selected_columns = filtered_metadata.iloc[:, 0].tolist()  # First column contains the column names

# Filter the metagenomic dataset for the selected columns
df = metagenomic_data[selected_columns]

# Select only the columns that have at least 100 non-null/non-zero values
df_species = df.loc[:, df.astype(bool).sum(axis=0) >= 100]

# Filter the metadata to keep only the rows that correspond to the columns of df_species
metadata = filtered_metadata[filtered_metadata.iloc[:, 0].isin(df_species.columns)]

# Transpose the dataframe and replace zeros with NaN
df_transposed = df_species.transpose()
df_transposed.replace(0, np.nan, inplace=True)

# Convert the dataframe to a numpy array
data = df_transposed.to_numpy()
print(data)

[[0.06894     nan 0.12161 ...     nan     nan     nan]
 [0.02226 2.66559 7.14212 ...     nan     nan     nan]
 [0.30753     nan 5.67888 ...     nan     nan     nan]
 ...
 [0.27213     nan 2.13695 ...     nan     nan     nan]
 [0.05766 1.09834 0.4529  ...     nan     nan     nan]
 [0.04521 0.19255 0.31595 ...     nan     nan     nan]]


In [12]:
def pl(x, lam, r):
    return(np.exp**(-r*x) + x^(-lam))



In [24]:
import numpy as np
from lmfit import Model

# get data into numpy ndarrays.  If in a simple data file, 
# with columns of data, that might look like:
data = df_transposed
p   = data[0, :]
s_u = data[1, :]

# define your model function (independent var in first argument)
def mod_su(p, su0=1, alpha=1):  # (values used as starting guesss)
    return su0 * (p)**(-alpha)

# now define the fitting model
model = Model(mod_su)

# make a set of parameters (for 'su0' and 'alpha'):
params = model.make_params(su0=10)  # can also set initial values here

# optionally, put min/max bounds on parameters:
params['alpha'].min = 0.0
params['su0'].min = 0.0
params['su0'].max = 1e6

# run the fit with Model.fit(Data_Array, Parameters, independent vars)
result = model.fit(s_u, params, p=p)

# print report with results and fitting statistics
print(result.fit_report())

# plot data and best fit
import matplotlib as plt
plt.plot(p, s_u, label='data')
plt.plot(p, result.best_fit, label='fit')
plt.show()

KeyError: (0, slice(None, None, None))